In [1]:
import os
import pandas as pd
import shutil as shu
import torchvision.datasets as ds
import torchvision.transforms as tt
import torch.utils.data as tud
from torchvision import models
from torch.autograd import Variable
import torch
from torch.optim import lr_scheduler
import time

In [78]:

pic_labels=pd.read_csv('F:/Kaggle_version2/bronchitis_detection/train_labels/Train_Labels.csv')
file_names=pic_labels[pic_labels['Labels']==0]['Images'].values

for i in os.listdir('F:/Kaggle_version2/bronchitis_detection/Train_Images'):
    if i.endswith ('.png'):
        if i[:-4] in file_names:
            shu.move(os.path.join(source_path,i),'F:/Kaggle_version2/bronchitis_detection/Train_Images/0')
            
        elif i[:-4] in file_names:
            shu.move(os.path.join(source_path,i),'F:/Kaggle_version2/bronchitis_detection/Train_Images/1')
           
    
    


In [89]:
normalize = tt.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
transforms=tt.Compose([tt.Resize(1000),
                               tt.CenterCrop(224),
                               tt.ToTensor(),
                               normalize])
data_folder=ds.ImageFolder('F:/Kaggle_version2/bronchitis_detection/Train_Images',transform=transforms)

In [90]:
dataloader=tud.DataLoader(data_folder,batch_size=20,shuffle=True,num_workers=2)

In [91]:
resnet = models.resnet50(pretrained=True)

In [59]:
inputs, labels = next(iter(dataloader))
#print (labels)
inputs, labels = Variable(inputs), Variable(labels)
#print (labels)

# for i, (images, labels) in enumerate(data_folder):
#     images = Variable(images)
#     labels = Variable(labels)

In [ ]:
outputs = resnet(inputs)
outputs.size()

In [92]:
def train_model(dataloders, model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    
    best_model_wts = model.state_dict()
    best_acc = 0.0
    dataset_sizes = {'train': len(dataloders['train'].dataset)}

    for epoch in range(num_epochs):
        for phase in ['train']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloders[phase]:
                inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
                train_epoch_acc = running_corrects / dataset_sizes[phase]
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                valid_epoch_acc = running_corrects / dataset_sizes[phase]
                
            if phase == 'valid' and valid_epoch_acc > best_acc:
                best_acc = valid_epoch_acc
                best_model_wts = model.state_dict()

        print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f} '.format(epoch, num_epochs - 1,train_epoch_loss, train_epoch_acc))
            
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [94]:
resnet = models.resnet50(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 2 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs,50)
# resnet.fc = torch.nn.Linear(num_ftrs,25)
resnet.fc = torch.nn.Linear(num_ftrs,10)
resnet.fc = torch.nn.Linear(num_ftrs,3)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

dloaders = {'train':dataloader}

In [95]:
start_time = time.time()
model = train_model(dloaders, resnet, criterion, optimizer, exp_lr_scheduler, num_epochs=100)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

Epoch [0/99] train loss: 0.0391 acc: 0.4531 
Epoch [1/99] train loss: 0.0356 acc: 0.5078 
Epoch [2/99] train loss: 0.0346 acc: 0.5578 
Epoch [3/99] train loss: 0.0354 acc: 0.5641 
Epoch [4/99] train loss: 0.0332 acc: 0.5859 
Epoch [5/99] train loss: 0.0334 acc: 0.5922 
Epoch [6/99] train loss: 0.0323 acc: 0.6266 
Epoch [7/99] train loss: 0.0315 acc: 0.6562 
Epoch [8/99] train loss: 0.0306 acc: 0.6672 
Epoch [9/99] train loss: 0.0307 acc: 0.6578 
Epoch [10/99] train loss: 0.0300 acc: 0.7000 
Epoch [11/99] train loss: 0.0309 acc: 0.6547 
Epoch [12/99] train loss: 0.0302 acc: 0.6875 
Epoch [13/99] train loss: 0.0302 acc: 0.6906 
Epoch [14/99] train loss: 0.0301 acc: 0.7016 
Epoch [15/99] train loss: 0.0298 acc: 0.7125 
Epoch [16/99] train loss: 0.0303 acc: 0.6953 
Epoch [17/99] train loss: 0.0304 acc: 0.7078 
Epoch [18/99] train loss: 0.0308 acc: 0.6609 
Epoch [19/99] train loss: 0.0299 acc: 0.7141 
Epoch [20/99] train loss: 0.0302 acc: 0.7125 
Epoch [21/99] train loss: 0.0304 acc: 0.7094

In [85]:
dloaders = {'train':dataloader}
dataset_sizes = {'train': len(dloaders['train'].dataset)}

In [26]:
dataset_sizes

{'train': 640}

In [96]:
normalize = tt.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
transforms=tt.Compose([tt.Resize(1000),
                               tt.CenterCrop(224),
                               tt.ToTensor(),
                               normalize])
data_folder=ds.ImageFolder('F:/Kaggle_version2/bronchitis_detection/test/',transform=transforms)

In [97]:
dataloader=tud.DataLoader(data_folder,shuffle=False,num_workers=2)

In [98]:
for inputs,labels in dataloader:
    #print (inputs)
    inputs=Variable(inputs)
    outputs = model(inputs)
    _, preds = torch.max(outputs.data, 1)
    print (preds)



 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 1
[torch.LongTensor of size 1]


 1
[torch.Lon

In [48]:
a=os.listdir('F:/Kaggle_version2/bronchitis_detection/test/Test_Images')

In [49]:
len(a
   )

160

In [51]:
len(set(a))

160

In [52]:
a


['CR_0001.png',
 'CR_0003.png',
 'CR_0004.png',
 'CR_0020.png',
 'CR_0022.png',
 'CR_0024.png',
 'CR_0026.png',
 'CR_0029.png',
 'CR_0030.png',
 'CR_0034.png',
 'CR_0035.png',
 'CR_0037.png',
 'CR_0041.png',
 'CR_0042.png',
 'CR_0047.png',
 'CR_0052.png',
 'CR_0068.png',
 'CR_0070.png',
 'CR_0071.png',
 'CR_0083.png',
 'CR_0096.png',
 'CR_0100.png',
 'CR_0101.png',
 'CR_0110.png',
 'CR_0111.png',
 'CR_0114.png',
 'CR_0121.png',
 'CR_0126.png',
 'CR_0131.png',
 'CR_0134.png',
 'CR_0137.png',
 'CR_0143.png',
 'CR_0145.png',
 'CR_0146.png',
 'CR_0148.png',
 'CR_0157.png',
 'CR_0159.png',
 'CR_0170.png',
 'CR_0173.png',
 'CR_0177.png',
 'CR_0182.png',
 'CR_0203.png',
 'CR_0208.png',
 'CR_0211.png',
 'CR_0214.png',
 'CR_0216.png',
 'CR_0219.png',
 'CR_0220.png',
 'CR_0233.png',
 'CR_0236.png',
 'CR_0240.png',
 'CR_0245.png',
 'CR_0247.png',
 'CR_0265.png',
 'CR_0266.png',
 'CR_0268.png',
 'CR_0283.png',
 'CR_0300.png',
 'CR_0304.png',
 'CR_0311.png',
 'CR_0313.png',
 'CR_0315.png',
 'CR_031